In [12]:
!pip3 install api.video

In [2]:
API_VIDEO_API_KEY = "dHGElV8BsOEzcjGM1eLOy5LYnrx0TyqR45TgfjMBPL2"

In [3]:
import apivideo
from apivideo.api import videos_api
from apivideo.model.video_update_payload import VideoUpdatePayload
from apivideo.model.bad_request import BadRequest
from apivideo.model.not_found import NotFound
from apivideo.model.metadata import Metadata
import json
import pandas as pd


def read_and_process_csv(file_path):
    df = pd.read_csv(file_path)
    df[["video_id", "activity", "sequence_id"]] = df["id"].str.split(
        "_", n=2, expand=True
    )
    df["sequence_id"] = df["sequence_id"].astype(str)

    df = df.rename(
        columns={
            "segment_id_sequence_start": "segment_id_sequence_start",
            "segment_id_sequence_end": "segment_id_sequence_end",
            "summary": "summary",
            "title": "title",
            "segment_start_timestamp": "segment_start_timestamp",
            "segment_end_timestamp": "segment_end_timestamp",
            "segment_start_timestamp_in_seconds": "segment_start_timestamp_in_seconds",
            "segment_end_timestamp_in_seconds": "segment_end_timestamp_in_seconds",
            "activity_type": "activity_type",
            "activity_reasoning": "activity_reasoning",
            "target_person_type": "target_person_type",
            "target_person_reasoning": "target_person_reasoning",
            "Moment_url": "Moment_url",
        }
    )

    df["index"] = df["id"]
    df["type"] = df["activity"].str.lower()

    df = df.astype(str)

    columns = [
        "index",
        "sequence_id",
        "segment_id_sequence_start",
        "segment_id_sequence_end",
        "summary",
        "title",
        "segment_start_timestamp",
        "segment_end_timestamp",
        "segment_start_timestamp_in_seconds",
        "segment_end_timestamp_in_seconds",
        "video_id",
        "activity_type",
        "activity_reasoning",
        "target_person_type",
        "target_person_reasoning",
        "activity",
        "Moment_url",
        "type",
    ]

    df = df[columns]

    return df


def update_video_metadata(api_instance, video_id, activities):
    try:
        video = api_instance.get(video_id)
        current_metadata = video.metadata

        current_metadata = [m for m in current_metadata if m.key != "activities"]

        new_activities = Metadata(
            key="activities", value=json.dumps(activities.to_dict("records"))
        )
        current_metadata.append(new_activities)

        update_payload = VideoUpdatePayload(metadata=current_metadata)

        api_instance.update(video_id, update_payload)
        print(f"Updated metadata for video {video_id}")
        return True
    except apivideo.ApiException as e:
        print(f"Exception when updating video {video_id}: {e}")
        return False


def main():
    df = read_and_process_csv("./activity_moments_clean_v3.csv")
    grouped_df = df.groupby("video_id")

    with apivideo.AuthenticatedApiClient(API_VIDEO_API_KEY) as api_client:
        api_instance = videos_api.VideosApi(api_client)

        for video_id, activities in grouped_df:
            success = update_video_metadata(api_instance, video_id, activities)
            if not success:
                print(f"Failed to update metadata for video {video_id}")

    print("Metadata update process completed.")


main()

Updated metadata for video vi18qSrl7wIdeYOKuDg7YQBx
Updated metadata for video vi1GPAuvhakg74TUpNoMMUvc
Updated metadata for video vi31gHvd0MMz8FonUstjUyMe
Updated metadata for video vi3MC0JsaO4n4cA7cYgYzpb4
Updated metadata for video vi42vP3lFwcaZP6hH82M5wZE
Updated metadata for video vi4flB6clm0iow6IFTbC7emI
Updated metadata for video vi5lUKF5SojsvO2eH5MdSXVs
Updated metadata for video vi6TBTdOQ3bkW2b7ku6xtvbK
Updated metadata for video vi6aqbh5ikJLOYkqOdO7TvN1
Updated metadata for video vi7ITVcbI2cXqYj2CnwKvSXg
Metadata update process completed.
